In [1]:
import sys
sys.path.append('../')
import globals as gl
import numpy as np
import topologies.DDF as DDF
import LP_gurobi as LP

# config=(6,2)
config=(36, 5)
EPR=3
topo='ddf'
_network=DDF.DDFtopo(config[0], config[1])
edge_list=list(_network.nx_graph.edges())

# Uniform traffic

In [2]:
p2p_TM=gl.generate_uniform_traffic_pattern(config[0], EPR)
R2R_TM=gl.convert_p2p_traffic_matrix_to_R2R(p2p_TM, config[0], EPR)

### DDF Benchmark:

In [3]:
results, _, path_dict = gl.calculate_DDF_routing(_network, config)
link_loads, local_link_load=_network.distribute_arbitrary_flow_on_paths_with_EPs(path_dict, EPR, p2p_TM)
print(f'predicted saturation load (DDF benchmark) = {local_link_load/max(link_loads)}')

calculation done for (36, 5) with unipath routing
predicted saturation load (DDF benchmark) = 0.6993464052287581


### Using shorter-than-3 paths

In [4]:
results, _, path_dict = gl.calculate_data_paths_within_length(_network, config, 3)
link_loads, local_link_load=_network.distribute_arbitrary_flow_on_paths_with_EPs(path_dict, EPR, p2p_TM)
print(f'predicted saturation load (before optimization) = {local_link_load/max(link_loads)}')
display(results)
all_weighted_paths, result_link_loads=LP.Solve_load_balancing(path_dict, edge_list,R2R_TM, _verbose=0)
link_loads, local_link_load=_network.distribute_arbitrary_flow_on_weighted_paths_with_EPs(all_weighted_paths, EPR, p2p_TM)
print(f'predicted saturation load (after optimization) = {local_link_load/max(link_loads)}')

calculation done for (36, 5) with shorter-than-3 paths routing
predicted saturation load (before optimization) = 0.519543578538478


{'diameter': 3,
 'ave_path_length_statistics': [1.0, 2.7314285714285713, 3.0],
 'num_paths_statistics': [1, 2.8285714285714287, 6],
 'link_load_statistics': [15.53333333333333,
  19.119999999999997,
  22.883333333333336]}

Set parameter Username
Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2411299


GurobiError: Token expired

DANR

In [ ]:
all_weighted_paths=gl.convert_path_dict_to_weighted_path_dict(path_dict)
link_loads, local_link_load=_network.distribute_arbitrary_flow_on_weighted_paths_with_EPs_return_dict(
                                        all_weighted_paths, EPR, p2p_TM)
all_paths_cost={}
for (s, d), weighted_paths in all_weighted_paths.items():
    all_paths_cost[(s, d)] = []
    for path, _ in weighted_paths:
        cost = 0 # this is the max edge load
        for i in range(len(path)-1):
            if cost < link_loads[(path[i], path[i+1])]:
                cost=link_loads[(path[i], path[i+1])]
        all_paths_cost[(s, d)].append((path, cost*len(path)))


num_iterations=1000

# every time maximally this amount of path probability 
# is transfered from the max cost path to the min cost path
# assume the link load is updated on real time for all routers
max_prob_transfer=0.1

max_link_load=max(link_loads.values())

for it in range(num_iterations):
    for (s, d), paths_cost in all_paths_cost.items():
        if len(all_weighted_paths[(s, d)])==1:
            continue

        max_path_cost=0
        max_cost_path_enum=-1
        min_path_cost=np.inf
        min_cost_path_enum=-1
        for i, (path, cost) in enumerate(paths_cost):
            if cost > max_path_cost and all_weighted_paths[(s, d)][i][1]>0 and cost/len(path)==max_link_load:
                max_path_cost = cost
                max_cost_path_enum=i

            if cost < min_path_cost and all_weighted_paths[(s, d)][i][1]<1:
                min_path_cost = cost
                min_cost_path_enum=i

        if max_path_cost==min_path_cost or max_cost_path_enum == -1:
            continue
        assert(min_cost_path_enum >= 0)

        max_cost_path_prob=all_weighted_paths[(s, d)][max_cost_path_enum][1]
        min_cost_path_prob=all_weighted_paths[(s, d)][min_cost_path_enum][1]
        prob_to_transfer = min([max_prob_transfer, 
                            (1 - all_weighted_paths[(s, d)][min_cost_path_enum][1]),
                            all_weighted_paths[(s, d)][max_cost_path_enum][1]])
        
        max_cost_path=all_weighted_paths[(s, d)][max_cost_path_enum][0]
        min_cost_path=all_weighted_paths[(s, d)][min_cost_path_enum][0]

        all_weighted_paths[(s, d)][max_cost_path_enum]=(max_cost_path, max_cost_path_prob-prob_to_transfer)
        all_weighted_paths[(s, d)][min_cost_path_enum]=(min_cost_path, min_cost_path_prob+prob_to_transfer)
        assert(0<=all_weighted_paths[(s, d)][max_cost_path_enum][1]<=1)
        assert(0<=all_weighted_paths[(s, d)][min_cost_path_enum][1]<=1)

        for i in range(len(max_cost_path)-1):
            link_loads[(max_cost_path[i], max_cost_path[i+1])]-=R2R_TM[s][d]*prob_to_transfer
        for i in range(len(min_cost_path)-1):
            link_loads[(min_cost_path[i], min_cost_path[i+1])]+=R2R_TM[s][d]*prob_to_transfer

        # link_loads, local_link_load=_network.distribute_arbitrary_flow_on_weighted_paths_with_EPs_return_dict(
        #                                         all_weighted_paths, EPR, p2p_TM)
        max_link_load=max(link_loads.values())
        print(f"at it{it}, sat load={local_link_load/max(link_loads.values())}")
        for (s, d), weighted_paths in all_weighted_paths.items():
            all_paths_cost[(s, d)] = []
            for path, _ in weighted_paths:
                cost = 0 # this is the max edge load
                for i in range(len(path)-1):
                    if cost < link_loads[(path[i], path[i+1])]:
                        cost=link_loads[(path[i], path[i+1])]
                all_paths_cost[(s, d)].append((path, cost*len(path)))


at it0, sat load=0.5195435785384787
at it0, sat load=0.5191654536632706
at it0, sat load=0.5195435785384789
at it0, sat load=0.5195435785384792
at it0, sat load=0.5195435785384794
at it0, sat load=0.5195435785384794
at it0, sat load=0.5195435785384795
at it0, sat load=0.5195435785384795
at it0, sat load=0.5172830553541208
at it0, sat load=0.5172830553541208
at it0, sat load=0.5195435785384797
at it0, sat load=0.5191654536632687
at it0, sat load=0.5195435785384797
at it0, sat load=0.5195435785384798
at it0, sat load=0.5195435785384798
at it0, sat load=0.5195435785384798
at it0, sat load=0.5195435785384802
at it0, sat load=0.5195435785384802
at it0, sat load=0.5195435785384807
at it0, sat load=0.5195435785384808
at it0, sat load=0.5214424951267039
at it0, sat load=0.5214424951267039
at it0, sat load=0.5195435785384808
at it1, sat load=0.521442495126704
at it1, sat load=0.5214424951267042
at it1, sat load=0.5214424951267045
at it1, sat load=0.5195435785384798
at it1, sat load=0.5214424951